In [1]:
!pip install langchain_huggingface

  Using cached langchain_huggingface-0.0.3-py3-none-any.whl (17 kB)
  Using cached huggingface_hub-0.23.4-py3-none-any.whl (402 kB)
  Using cached langchain_core-0.2.12-py3-none-any.whl (355 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
  Using cached transformers-4.42.3-py3-none-any.whl (9.3 MB)
  Using cached tokenizers-0.19.1-cp310-none-win_amd64.whl (2.2 MB)
  Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl (145 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
  Using cached filelock-3.15.4-py3-none-any.whl (16 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached langsmith-0.1.84-py3-none-any.whl (127 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl (28 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl (423 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl (7.6 kB)
  Using cached orjson-3.10.6-cp31

You should consider upgrading via the 'E:\HealthyLungs\hack\Scripts\python.exe -m pip install --upgrade pip' command.
